<div style="direction:rtl; font-size:12px">به نام خدا</div>
<div style="direction:rtl; font-size:12px">گروه 8 : آرمین افتخاری - محمد سلیمان بهزاد - محمدرضا پوررضا - رضا برزگر طرقبه - سبحان مرادیان دقیق</div>
<div style="direction:rtl"> در این بخش با استفاده از الگوریتم KNN که یک کلاسیفایر رایج در مبحث ماشین لرنینگ می باشد استفاده کرده ایم. با استفاده از این الگوریتم می توانیم داده های train خود را کلاستر بندی کنیم. سپس مدلی ترین کنیم و با استفاده از آن به ازای هر داده تست، یک همسایگی 10 تایی از شبیه ترین داده ها را پیدا می کنیم. توضیحات بیشتر و جزئی تر در داکیومنت و خلال کد ها داده شده است.</div>

### Implementation of KNN

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors , KNeighborsClassifier
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


### Reading the dataset which was preprocessed

In [2]:
book_ratings = pd.read_csv('preprocessed_ratings.csv')

In [3]:
book_ratings.head()

,User_ID,ISBN,Book_Rating,Book_Title,Book-Author,Year_Of_Publication,Publisher,Location,Age
0,53,451,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,"strafford, missouri, usa",34.0
1,53,280,0,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown","strafford, missouri, usa",34.0
2,53,647,0,The Da Vinci Code,Dan Brown,2003,Doubleday,"strafford, missouri, usa",34.0
3,53,2028,0,Wild Animus,Rich Shapero,2004,Too Far,"strafford, missouri, usa",34.0
4,53,241,0,Four To Score (A Stephanie Plum Novel),Janet Evanovich,1999,St. Martin's Paperbacks,"strafford, missouri, usa",34.0


In [4]:
books = book_ratings.filter(['Book_Title' , 'Book-Author','Publisher']).copy()
books = books.drop_duplicates(subset='Book_Title', keep='first')

### Make ratings matrix 

In [5]:
# dataset_pivot = book_ratings.pivot(index='ISBN', columns='User_ID', values='Book_Rating').fillna(0)

In [6]:
# sim_matrix = csr_matrix(dataset_pivot.values)

In [7]:
 with open('sim_matrix', 'rb') as fp:
    sim_matrix = pickle.load(fp)

In [8]:
# sum_of_squared_distances = []
# K = range(1,100)

# for k in K:
#     km = KMeans(n_clusters=k, max_iter=5, n_init=10)
#     km = km.fit(sim_matrix)
#     sum_of_squared_distances.append(km.inertia_)

# plt.plot(K, sum_of_squared_distances, 'bx-')
# plt.xlabel('k')
# plt.ylabel('sum_of_squared_distances')
# plt.title('Elbow Method For Optimal k')
# plt.show()

In [9]:
true_k = 7
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=50)
model.fit(sim_matrix)
labels=model.labels_
books['label'] = labels

In [10]:
groups = books.groupby(by=['label'])
groups.first()

,Book_Title,Book-Author,Publisher
label,,,
0,Flesh Tones: A Novel,M. J. Rose,Ballantine Books
1,The Lovely Bones: A Novel,Alice Sebold,"Little, Brown"
2,The Green Mile,Stephen King,Pocket
3,I'll Be Seeing You,Mary Higgins Clark,Pocket
4,Tara Road,Maeve Binchy,Dell Publishing Company
5,Roses Are Red (Alex Cross Novels),James Patterson,Warner Vision
6,Q Is for Quarry,Sue Grafton,Putnam Publishing Group


In [11]:
for label, pdf in groups:
    print("\n\nlabel: ", label, "    counts: ", len(pdf))
    print("book title: ", pdf['Book_Title'], "    book author: ",pdf['Book-Author'] ,  "    book publisher: ",pdf['Publisher'] )





label:  0     counts:  311
book title:  0                                     Flesh Tones: A Novel
19           The Dress Lodger (Ballantine Reader's Circle)
62       Big Stone Gap: A Novel (Ballantine Reader's Ci...
69                                             The Surgeon
71                                         The Murder Book
                               ...                        
10726                                        The Straw Men
11256                             Secret History : A Novel
12339     The Main Corpse (Culinary Mysteries (Paperback))
13226        The Orchid Thief (Ballantine Reader's Circle)
20246                                             Creature
Name: Book_Title, Length: 311, dtype: object     book author:  0                 M. J. Rose
19              Sheri Holman
62          Adriana Trigiani
69            TESS GERRITSEN
71        JONATHAN KELLERMAN
                ...         
10726       Michael Marshall
11256            DONNA TARTT
12339    Diane

### Split the dataset into train and test

In [12]:
train_data, test_data = train_test_split(books, test_size=0.20)
train_X = train_data.drop(['label'],axis=1)
train_Y = train_data['label']

test_X = test_data.drop(['label'],axis=1)
test_Y = test_data['label']

In [13]:
encoder = LabelEncoder()
for col in train_X.columns :
    train_X[col] = encoder.fit_transform(train_X[col])
    
for col in test_X.columns :
    test_X[col] = encoder.fit_transform(test_X[col])


### Make ratings matrix 

In [14]:
# train_data_pivot = train_data.pivot(index='ISBN', columns='User_ID', values='Book_Rating').fillna(0)

### Fill in the columns which ignored in the ratings matrix, in order to have the same dimensionality with original matrix

In [15]:
# diff_cols_train = list(set(range(2954)) - set(train_data_pivot.columns))
# for user in diff_cols_train:
#     train_data_pivot[str(user)] = 0

In [16]:
# train_data_matrix = csr_matrix(train_data_pivot.values)

### Do the same for test dataset

In [17]:
# test_data_pivot = test_data.pivot(index='ISBN', columns='User_ID', values='Book_Rating').fillna(0)

In [18]:
# diff_cols_test = list(set(range(2954)) - set(test_data_pivot.columns))
# for user in diff_cols_test:
#     test_data_pivot[str(user)] = 0

### Make a KNN model and fit it

In [19]:

# knn = NearestNeighbors(metric='cosine', algorithm='brute')
# knn.fit(train_data_matrix)

In [25]:
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(train_X, train_Y)
predict = knn.predict(test_X)
accuracy_score(test_Y, predict)


0.597911227154047

### Ok, now try to make recommendations for an input row of test dataset

In [21]:
query_index = 37 # For examle
distance, indices = knn.kneighbors(test_data_pivot.iloc[query_index, :].values.reshape(1, -1), n_neighbors= 11)

for i in range(len(distance.flatten())):
    if i == 0:
        print("Recommendation for {0}:\n".format(book_ratings[book_ratings['ISBN'] == train_data_pivot.index[query_index]]['Book_Title'].unique()))
    else:
        print("{0}: {1}".format(i, book_ratings[book_ratings['ISBN'] == train_data_pivot.index[indices.flatten()[i]]]['Book_Title'].unique()))

NameError: name 'test_data_pivot' is not defined